# Pandas for Data Analysis

## Objectives

After completing this lab you will be able to:

* load CSV data into pandas DataFrames
* inspect, clean, and transform columns
* filter, group, and summarize data
* merge datasets for deeper analysis
* answer analytical questions with pandas

<h2>Table of Contents</h2>
<div class="alert alert-block alert-info" style="margin-top: 20px">
    <ul>
        <li>
            <a href="#intro">Introduction to pandas</a>
        </li>
        <li>
            <a href="#load">Load the CSV files</a>
        </li>
        <li>
            <a href="#inspect">Inspect and understand the data</a>
        </li>
        <li>
            <a href="#clean">Clean and prepare columns</a>
        </li>
        <li>
            <a href="#filter">Filter and sort</a>
        </li>
        <li>
            <a href="#group">Group and aggregate</a>
        </li>
        <li>
            <a href="#join">Join datasets</a>
        </li>
        <li>
            <a href="#ex">Exercises</a>
        </li>
    </ul>
</div>

<hr>

<h2 id="intro">Introduction to pandas</h2>

pandas is a Python library for working with tabular data. It provides two main data structures:

* **Series**: a one-dimensional labeled array
* **DataFrame**: a two-dimensional labeled table

In [ ]:
import pandas as pd

pd.__version__

<h2 id="load">Load the CSV files</h2>

The CSV files are in the same folder as this notebook. We will load them into DataFrames.

In [ ]:
df_crime = pd.read_csv('ChicagoCrimeData.csv')
df_schools = pd.read_csv('ChicagoPublicSchools.csv')
df_census = pd.read_csv('ChicagoCensusData.csv')

df_crime.shape, df_schools.shape, df_census.shape

In [ ]:
df_crime.head()

In [ ]:
df_schools.head()

In [ ]:
df_census.head()

<h2 id="inspect">Inspect and understand the data</h2>

Use `.info()` to see data types and missing values, and `.describe()` for quick numeric summaries.

In [ ]:
df_crime.info()

In [ ]:
df_schools.info()

In [ ]:
df_census.describe()

<h2 id="clean">Clean and prepare columns</h2>

We will standardize column names and clean common missing-value markers like `NDA`.

In [ ]:
def standardize_columns(df):
    df.columns = (
        df.columns
        .str.strip()
        .str.replace(r'[^\w]+', '_', regex=True)
        .str.strip('_')
        .str.lower()
    )
    return df

df_crime = standardize_columns(df_crime)
df_schools = standardize_columns(df_schools)
df_census = standardize_columns(df_census)

df_schools = df_schools.rename(columns={'elementary_middle_or_high_school': 'school_type'})
df_schools.columns[:10]

In [ ]:
for df in [df_schools, df_census]:
    df.replace('NDA', pd.NA, inplace=True)

numeric_cols = [
    'safety_score',
    'family_involvement_score',
    'environment_score',
    'instruction_score',
    'leaders_score',
    'teachers_score',
    'parent_engagement_score',
    'parent_environment_score',
    'average_student_attendance',
    'average_teacher_attendance',
    'rate_of_misconducts__per_100_students_',
]

for col in numeric_cols:
    if col in df_schools.columns:
        df_schools[col] = pd.to_numeric(df_schools[col].astype(str).str.replace('%', ''), errors='coerce')

<h2 id="filter">Filter and sort</h2>

Find schools with high safety scores and sort by the score.

In [ ]:
high_safety = df_schools[df_schools['safety_score'] >= 90]
high_safety = high_safety[['name_of_school', 'school_type', 'safety_score', 'community_area_name']]
high_safety.sort_values('safety_score', ascending=False).head(10)

<h2 id="group">Group and aggregate</h2>

Count crimes by year and compute arrest rates.

In [ ]:
crime_by_year = df_crime.groupby('year').size().reset_index(name='crime_count')
crime_by_year.sort_values('year').head()

In [ ]:
arrest_rate = (
    df_crime.groupby('year')['arrest']
    .mean()
    .reset_index(name='arrest_rate')
)
arrest_rate.head()

A quick crosstab of arrests by year:

In [ ]:
pd.crosstab(df_crime['year'], df_crime['arrest']).head()

<h2 id="join">Join datasets</h2>

We can join schools and census data by `community_area_number` to compare school quality and hardship.

In [ ]:
schools_census = df_schools.merge(
    df_census[['community_area_number', 'community_area_name', 'hardship_index']],
    on='community_area_number',
    how='left',
)

schools_census[['name_of_school', 'community_area_name', 'hardship_index']].head()

In [ ]:
schools_census.groupby('hardship_index')['safety_score'].mean().reset_index().head()

<h2 id="ex">Exercises</h2>

Use pandas to answer the following questions.

1) How many rows and columns are in each dataset? Return a small table with dataset name, rows, and columns.

In [ ]:
# Write your code below

<details><summary>Click here for the solution</summary>

```python
summary = pd.DataFrame(
    {
        'dataset': ['crime', 'schools', 'census'],
        'rows': [len(df_crime), len(df_schools), len(df_census)],
        'columns': [df_crime.shape[1], df_schools.shape[1], df_census.shape[1]],
    }
)
summary
```

</details>

2) What are the top 5 `primary_type` crime categories overall?

In [ ]:
# Write your code below

<details><summary>Click here for the solution</summary>

```python
df_crime['primary_type'].value_counts().head(5)
```

</details>

3) For the year 2011, compute the arrest rate by `primary_type` and show the top 5.

In [ ]:
# Write your code below

<details><summary>Click here for the solution</summary>

```python
(
    df_crime[df_crime['year'] == 2011]
    .groupby('primary_type')['arrest']
    .mean()
    .sort_values(ascending=False)
    .head(5)
    .reset_index(name='arrest_rate')
)
```

</details>

4) Which community areas have the highest hardship index? Show the top 10 with names.

In [ ]:
# Write your code below

<details><summary>Click here for the solution</summary>

```python
df_census[['community_area_name', 'hardship_index']].sort_values(
    'hardship_index', ascending=False
).head(10)
```

</details>

5) Among high schools only, what is the average `safety_score` by community area?

In [ ]:
# Write your code below

<details><summary>Click here for the solution</summary>

```python
(
    df_schools[df_schools['school_type'] == 'HS']
    .groupby('community_area_name')['safety_score']
    .mean()
    .reset_index()
    .sort_values('safety_score', ascending=False)
    .head(10)
 )
```

</details>

6) Join crime and census data. For each community area, compute crimes per year and list the 5 highest in 2011.

In [ ]:
# Write your code below

<details><summary>Click here for the solution</summary>

```python
crime_census = df_crime.merge(
    df_census[['community_area_number', 'community_area_name']],
    on='community_area_number',
    how='left',
 )
per_year = (
    crime_census
    .groupby(['community_area_number', 'community_area_name', 'year'])
    .size()
    .reset_index(name='crime_count')
 )
per_year[per_year['year'] == 2011].sort_values(
    'crime_count', ascending=False
).head(5)
```

</details>

7) Create a pivot table of crime counts by `primary_type` (rows) and `year` (columns) for years 2009-2011.

In [ ]:
# Write your code below

<details><summary>Click here for the solution</summary>

```python
subset = df_crime[df_crime['year'].between(2009, 2011)]
subset.pivot_table(
    index='primary_type',
    columns='year',
    values='id',
    aggfunc='count',
    fill_value=0,
 )
```

</details>

8) Is there a relationship between `hardship_index` and `safety_score`? Compute the correlation.

In [ ]:
# Write your code below

<details><summary>Click here for the solution</summary>

```python
schools_census[['hardship_index', 'safety_score']].dropna().corr().loc[
    'hardship_index', 'safety_score'
 ]
```

</details>